<a href="https://colab.research.google.com/github/madhesh60/retail/blob/main/extractjson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install onnxruntime
from rembg import remove
from PIL import Image
from google.colab import files
import io
import numpy as np

print("Upload your PRODUCT image:")
uploaded = files.upload()

product_filename = next(iter(uploaded))
product_img = Image.open(io.BytesIO(uploaded[product_filename]))

product_cutout = remove(product_img)
product_cutout_path = "product_cutout.png"
product_cutout.save(product_cutout_path)

files.download(product_cutout_path)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.5 MB/s eta 0:00:00
Upload your PRODUCT image:


Saving Screenshot 2025-12-03 103253.png to Screenshot 2025-12-03 103253.png


100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 71.8GB/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# Load cutout as RGBA
product_cutout = Image.open(product_cutout_path).convert("RGBA")
alpha = product_cutout.split()[-1]
alpha_np = np.array(alpha)

product_mask_path = "product_mask.png"
alpha.save(product_mask_path)

# bounding box
ys, xs = np.where(alpha_np > 0)
bbox = {
    "x": int(xs.min()),
    "y": int(ys.min()),
    "width": int(xs.max() - xs.min()),
    "height": int(ys.max() - ys.min())
}

# aspect ratio
w = bbox["width"]
h = bbox["height"]
ratio = w / h

if ratio < 0.75:
    aspect = "tall"
elif ratio > 1.3:
    aspect = "wide"
else:
    aspect = "square"

bbox, aspect

({'x': 129, 'y': 19, 'width': 129, 'height': 347}, 'tall')

In [5]:
import cv2
from sklearn.cluster import KMeans

img = cv2.imread(product_cutout_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
pixels = img.reshape(-1, 3)

kmeans = KMeans(n_clusters=3)
kmeans.fit(pixels)
colors = kmeans.cluster_centers_.astype(int)

primary = colors[0].tolist()
secondary = colors[1].tolist()
accent = colors[2].tolist()

primary, secondary, accent

([0, 0, 0], [224, 226, 228], [70, 93, 119])

In [6]:
import easyocr
reader = easyocr.Reader(['en'])

ocr_text = reader.readtext(product_cutout_path, detail=0)
print("OCR Result:", ocr_text)

brand_name = ocr_text[0] if len(ocr_text) > 0 else ""
product_name = ocr_text[1] if len(ocr_text) > 1 else ""
variant = ocr_text[2] if len(ocr_text) > 2 else ""
claims = ocr_text[3:] if len(ocr_text) > 3 else []

brand_name, product_name, variant, claims


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


OCR Result: ['heade', 'shoukders', 'PURIFICACION', 'CAPILAR', 'E5"', 'Ioo', '375 mI']


('heade', 'shoukders', 'PURIFICACION', ['CAPILAR', 'E5"', 'Ioo', '375 mI'])

In [7]:
print("Upload your LOGO image:")
uploaded_logo = files.upload()

logo_filename = next(iter(uploaded_logo))
logo_img = Image.open(io.BytesIO(uploaded_logo[logo_filename]))

logo_cutout = remove(logo_img)
logo_cutout_path = "logo_cutout.png"
logo_cutout.save(logo_cutout_path)

files.download(logo_cutout_path)

Upload your LOGO image:


Saving Screenshot 2025-12-03 120747.png to Screenshot 2025-12-03 120747.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
# mask
logo_cutout = Image.open(logo_cutout_path).convert("RGBA")
logo_alpha = logo_cutout.split()[-1]
logo_alpha_np = np.array(logo_alpha)

logo_mask_path = "logo_mask.png"
logo_alpha.save(logo_mask_path)

# bounding box
lys, lxs = np.where(logo_alpha_np > 0)
logo_bbox = {
    "x": int(lxs.min()),
    "y": int(lys.min()),
    "width": int(lxs.max() - lxs.min()),
    "height": int(lys.max() - lys.min())
}

# colors
logo_img_cv = cv2.imread(logo_cutout_path)
logo_img_cv = cv2.cvtColor(logo_img_cv, cv2.COLOR_BGR2RGB)
logo_pixels = logo_img_cv.reshape(-1,3)

kmeans_logo = KMeans(n_clusters=2)
kmeans_logo.fit(logo_pixels)
logo_colors = kmeans_logo.cluster_centers_.astype(int)

logo_primary = logo_colors[0].tolist()
logo_secondary = logo_colors[1].tolist()

logo_bbox, logo_primary, logo_secondary


({'x': 4, 'y': 0, 'width': 404, 'height': 254}, [159, 168, 189], [6, 9, 16])

In [9]:
import pdfplumber
import re

print("Upload BRAND GUIDELINE PDF:")
uploaded_pdf = files.upload()

pdf_name = next(iter(uploaded_pdf))
text = ""

with pdfplumber.open(io.BytesIO(uploaded_pdf[pdf_name])) as pdf:
    for page in pdf.pages:
        t = page.extract_text()
        if t: text += t + "\n"

Upload BRAND GUIDELINE PDF:


Saving Hex Colors.pdf to Hex Colors.pdf


In [10]:
# Colors
hex_colors = re.findall(r'#[0-9A-Fa-f]{6}', text)

# Fonts
fonts = re.findall(r'Font(?:s)?:\s*([A-Za-z0-9 ,\-]+)', text)

# Forbidden phrases
forbidden = re.findall(r'Forbidden Words:\s*(.*)', text)

# Logo rules
logo_min_size = re.findall(r'Minimum Logo Size:\s*(\d+)', text)
clear_space = re.findall(r'Clear Space:\s*([0-9.]+)', text)

In [11]:
print("Upload RETAILER GUIDELINES PDF:")
uploaded_pdf2 = files.upload()

pdf2_name = next(iter(uploaded_pdf2))
text2 = ""

with pdfplumber.open(io.BytesIO(uploaded_pdf2[pdf2_name])) as pdf:
    for page in pdf.pages:
        t = page.extract_text()
        if t: text2 += t + "\n"


Upload RETAILER GUIDELINES PDF:


Saving Hex Colors.pdf to Hex Colors (1).pdf


In [12]:
allowed_sizes = re.findall(r'(\d{3,4})x(\d{3,4})', text2)
max_file_size = re.findall(r'Max File Size:\s*(\d+)\s*KB', text2)

In [13]:
user_input = {
    "campaign_goal": "awareness",
    "main_message": "20% OFF",
    "cta_text": "Shop Now",
    "style_preference": "clean",
    "target_audience": "youth"
}
channel_specs = {
    "platform": "instagram",
    "placement": "story",
    "recommended_headline_length": 40,
    "recommended_subtext_length": 60,
    "safe_areas": {
        "top_percent": 10,
        "bottom_percent": 15
    }
}

In [14]:
import json

final_json = {
    "product_image": {
        "cutout_image_url": product_cutout_path,
        "mask_url": product_mask_path,
        "bounding_box": bbox,
        "aspect_ratio": aspect,
        "dominant_colors": {
            "primary": primary,
            "secondary": secondary,
            "accent": accent
        },
        "ocr_text": {
            "brand_name": brand_name,
            "product_name": product_name,
            "variant": variant,
            "claims": claims
        }
    },
    "logo": {
        "clean_logo_url": logo_cutout_path,
        "mask_url": logo_mask_path,
        "bounding_box": logo_bbox,
        "dominant_colors": {
            "primary": logo_primary,
            "secondary": logo_secondary
        }
    },
    "brand_guidelines": {
        "allowed_colors": hex_colors,
        "allowed_fonts": fonts,
        "logo_rules": {
            "min_width_px": int(logo_min_size[0]) if logo_min_size else 0,
            "clear_space_ratio": float(clear_space[0]) if clear_space else 0.0,
            "allow_color_change": False
        },
        "copy_rules": {
            "forbidden_phrases": forbidden,
            "tone": "",
            "allowed_phrases": [],
            "max_headline_chars": 50,
            "max_subtext_chars": 120
        }
    },
    "retailer_guidelines": {
        "allowed_sizes": allowed_sizes,
        "file_rules": {
            "allowed_formats": ["png","jpg"],
            "max_file_size_kb": int(max_file_size[0]) if max_file_size else 500
        }
    },
    "channel_specs": channel_specs,
    "user_input": user_input
}

with open("final_extraction.json","w") as f:
    json.dump(final_json, f, indent=2)

files.download("final_extraction.json")
print("FINAL JSON GENERATED SUCCESSFULLY!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FINAL JSON GENERATED SUCCESSFULLY!
